<a href="https://colab.research.google.com/github/YehorChykin/HW_DS/blob/main/HW_DS_M6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпортуємо

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Читаємо фрейм

In [37]:
df = pd.read_csv("/content/drive/MyDrive/DS_and_ML/Housing.csv")
print(df.head())
print(df.shape)

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
(545, 13)


напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [38]:
def hypothesis(theta, X):
    return np.dot(X, theta)

створіть функцію для обчислення функції втрат у векторному вигляді;

In [39]:
def compute_cost(theta, X, y):
    m = len(y)
    h = hypothesis(theta, X)
    J = np.sum((h - y)**2) / (2 * m)
    return J

реалізуйте один крок градієнтного спуску;

In [40]:
def gradient_descent_step(theta, X, y, learning_rate):
    m = len(y)  # Кількість прикладів у навчальному наборі
    h = hypothesis(theta, X)  # Обчислюємо гіпотезу лінійної регресії
    gradient = np.dot(X.T, (h - y)) / m  # Обчислюємо градієнт функції втрат
    theta -= learning_rate * gradient  # Оновлюємо параметри theta
    return theta

або:

In [41]:
def gradient_descent_step(theta, X, y, alpha):
    m = len(y)
    h = hypothesis(theta, X)
    gradient = np.dot(X.T, (h - y)) / m
    new_theta  = theta - alpha * gradient
    return new_theta


знайдіть найкращі параметри w⃗
 для датасету використовуючи написані вами функції, прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [55]:
# Вибираємо фічі та цільову змінну
X = df[['area', 'bedrooms', 'bathrooms']].values
y = df['price'].values

# Нормалізуємо дані
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

scaler_y = StandardScaler()
y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Додаємо стовпець з одиницями для параметра theta0
X_normalized = np.concatenate([np.ones((X_normalized.shape[0], 1)), X_normalized], axis=1)

def gradient_descent(theta, X, y, learning_rate, iterations):
    m = len(y)
    costs = []  # Зберігатимемо значення функції втрат на кожній ітерації

    for _ in range(iterations):
        # Обчислюємо гіпотезу
        h = hypothesis(theta, X)

        # Обчислюємо градієнт та оновлюємо параметри theta
        theta = gradient_descent_step(theta, X, y, learning_rate)

        # Обчислюємо функцію втрат та зберігаємо її значення
        cost = compute_cost(theta, X, y)
        costs.append(cost)

    return theta, costs

# Налаштуємо параметри градієнтного спуску
learning_rate = 0.00000001
iterations = 1000

# Запускаємо градієнтний спуск
theta_optimal, costs = gradient_descent(np.zeros(X_normalized.shape[1]), X_normalized, y, learning_rate, iterations)
# Денормалізація для градієнтного спуску
intercept_gradient_descent = scaler_y.inverse_transform(np.array([theta_optimal[0]]).reshape(1, -1))
print("Денормалізоване значення інтерсепту (градієнтний спуск):", intercept_gradient_descent[0][0])

# Виводимо оптимальні параметри theta
print("Оптимальні параметри theta:", theta_optimal)


Денормалізоване значення інтерсепту (градієнтний спуск): 93843241.91688022
Оптимальні параметри theta: [47.66705438 10.01624017  6.84869765  9.67141711]


знайдіть ці ж параметри за допомогою аналітичного рішення;

In [57]:
def normalize_data(X, y):
    # Нормалізуємо ознаки X
    scaler_X = StandardScaler()
    X_normalized = scaler_X.fit_transform(X)

    # Нормалізуємо цільову змінну y
    scaler_y = StandardScaler()
    y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

    return X_normalized, y_normalized

# Вибираємо фічі та цільову змінну
X = df[['area', 'bedrooms', 'bathrooms']].values
y = df['price'].values

# Нормалізуємо дані
X_normalized, y_normalized = normalize_data(X, y)

# Додаємо стовпець з одиницями для параметра theta0
X_normalized = np.concatenate([np.ones((X_normalized.shape[0], 1)), X_normalized], axis=1)

# Обчислюємо оптимальні параметри за допомогою аналітичного рішення
theta_optimal = np.linalg.inv(X_normalized.T.dot(X_normalized)).dot(X_normalized.T).dot(y_normalized)

# Денормалізація для аналітичного рішення
intercept_analytical_solution = scaler_y.inverse_transform(np.array([theta_optimal[0]]).reshape(1, -1))

print("Денормалізоване значення інтерсепту (аналітичний рішення):", intercept_analytical_solution[0][0])

print("Оптимальні параметри theta:", theta_optimal)


Денормалізоване значення інтерсепту (аналітичний рішення): 4766729.247706423
Оптимальні параметри theta: [6.82595372e-16 4.39452085e-01 1.60528660e-01 3.72344423e-01]


для перевірки спрогнозованих значень, використайте LinearRegression з бібліотеки scikit-learn та порівняйте результати.

In [67]:
# Розділимо дані на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створимо та навчимо модель лінійної регресії на навчальних даних
model = LinearRegression()
model.fit(X_train, y_train)

# Зробимо прогнози на тестових даних
y_pred = model.predict(X_test)

# Виведемо оптимальні параметри theta для порівняння
print("Оптимальні параметри theta (градієнтний спуск):", theta_optimal)
print("Оптимальні параметри theta (аналітичний рішення):", model.coef_, model.intercept_)

# Оцінимо точність моделі за допомогою середньо квадратичної помилки (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Середньо квадратична помилка (MSE):", mse)


Оптимальні параметри theta (градієнтний спуск): [6.82595372e-16 4.39452085e-01 1.60528660e-01 3.72344423e-01]
Оптимальні параметри theta (аналітичний рішення): [3.45466570e+02 3.60197650e+05 1.42231966e+06] 59485.379208717495
Середньо квадратична помилка (MSE): 2750040479309.0522


In [71]:
# Створимо та навчимо модель лінійної регресії на нормалізованих даних
model = LinearRegression()
model.fit(X_normalized, y_normalized)

# Отримаємо оптимальні параметри моделі
coef_normalized = model.coef_
intercept_normalized = model.intercept_

# Денормалізуємо параметри (інтерсепт)
intercept_denormalized = scaler_y.inverse_transform(np.array([intercept_normalized]).reshape(1, -1))

print("Оптимальні параметри theta (LinearRegression, нормалізовані):", coef_normalized)
print("Денормалізоване значення інтерсепту (LinearRegression):", intercept_denormalized[0][0])

# Передбачення на нормалізованих даних
y_pred_normalized = model.predict(X_normalized)

# Переведемо передбачення назад у вихідні одиниці
y_pred_denormalized = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, 1)).flatten()

# Обчислимо середньо квадратичну помилку (MSE) на нормалізованих даних
mse_normalized = mean_squared_error(y, y_pred_denormalized)
print("Середньо квадратична помилка (MSE) (LinearRegression, нормалізовані дані):", mse_normalized)


Оптимальні параметри theta (LinearRegression, нормалізовані): [0.         0.43945209 0.16052866 0.37234442]
Денормалізоване значення інтерсепту (LinearRegression): 4766729.247706423
Середньо квадратична помилка (MSE) (LinearRegression, нормалізовані дані): 1791170049977.3193


Ще раз

In [75]:



# Створимо та навчимо модель лінійної регресії на нормалізованих даних
model = LinearRegression()
model.fit(X_normalized, y_normalized)

# Отримаємо оптимальні параметри моделі
coef_normalized = model.coef_
intercept_normalized = model.intercept_

# Денормалізуємо параметри (інтерсепт)
intercept_denormalized = scaler_y.inverse_transform([[intercept_normalized]])[0]

print("Оптимальні параметри theta (LinearRegression, нормалізовані):", coef_normalized)
print("Денормалізоване значення інтерсепту (LinearRegression):", intercept_denormalized)

# Передбачення на нормалізованих даних
y_pred_normalized = model.predict(X_normalized)

# Переведемо передбачення назад у вихідні одиниці
y_pred_denormalized = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, 1)).flatten()

# Обчислимо середньо квадратичну помилку (MSE) на нормалізованих даних
mse_normalized = mean_squared_error(y, y_pred_denormalized)
print("Середньо квадратична помилка (MSE) (LinearRegression, нормалізовані дані):", mse_normalized)


Оптимальні параметри theta (LinearRegression, нормалізовані): [0.         0.43945209 0.16052866 0.37234442]
Денормалізоване значення інтерсепту (LinearRegression): [4766729.24770642]
Середньо квадратична помилка (MSE) (LinearRegression, нормалізовані дані): 1791170049977.3193
